<a href="https://colab.research.google.com/github/derrodo61/notebooks/blob/main/neo4j_create.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create using merge

## Create nodes

```
LOAD CSV WITH HEADERS
from 'https://raw.githubusercontent.com/MidnightSkyUniverse/udemyNeo4j/master/data/titles_and_skills.csv'
AS row
MERGE (t:Title {name: row.Title})
```

```
LOAD CSV WITH HEADERS
from 'https://raw.githubusercontent.com/MidnightSkyUniverse/udemyNeo4j/master/data/skills_details.csv'
AS row
MERGE (s:Skill {name: row.Skill})
SET
s.id = row.ID,
s.description = row.Description,
s.category=row.Category
```

## Create edges / relations

```
MATCH (t:Title)
UNWIND t.skills AS skill
MERGE (s:Skill {name: skill})
MERGE (t)-[:REQUIRES]->(s)
```

## Match

```
match (n) RETURN n
```

```
MATCH (n:Skill|Title) RETURN n
```
